#  US Census geocoding information 
Add location info (e.g. block groups/fips) to list of places (schools)

In [14]:
import os
import requests
import pandas as pd
import io
import csv

In [15]:
# filename = 'uscensus/data/School-Addresses.csv'
filename = '/home/sean/Downloads/Duval schools full list SoC.csv'
# filename = '/opt/work/wooly-tongue/src/jupyter/uscensus/data/School-AddressesNoMatch.csv'
dfDistrict = pd.read_csv(filename, sep=',', engine='python')

In [16]:
# import os
# # Get the current working directory
# current_directory = os.getcwd()
# 
# print(current_directory)
# dfDistrict = dfDistrict.dropna(how='any')# 

In [17]:
dfDistrict.shape

(381, 5)

In [18]:
dfDistrict.head()

,1,1157 Golfair Blvd,Jacksonville,Fl,32209-4037
0,2,555 Wonderwood Dr,Atlantic Beach,Fl,32233-4699
1,3,964 Saint Clair St,Jacksonville,Fl,32254-3148
2,4,1201 University Blvd N,Jacksonville,Fl,32211-8851
3,5,298 Sherry Dr,Atlantic Beach,Fl,32233-5238
4,6,3257 Lake Shore Blvd,Jacksonville,Fl,32210-5345


In [19]:
url = 'https://geocoding.geo.census.gov/geocoder/geographies/addressbatch'
payload = {'benchmark':'Public_AR_Current','vintage':'Current_Current'}

In [20]:
print(f"Prepare to query Census API using address file: {filename}")
files = {'addressFile': (filename, open(filename, 'rb'), 'text/csv')}
s = requests.post(url, files=files, data=payload, timeout=None)
contents = s.content
print(f"\t\tGot reponse from Census API: {s.status_code} -- Content length: {len(contents)}")

Prepare to query Census API using address file: /home/sean/Downloads/Duval schools full list SoC.csv
		Got reponse from Census API: 200 -- Content length: 74524


In [21]:
import io
csv_string_readable = io.StringIO(contents.decode('utf-8'))
csv_string_readable

In [22]:
contents
# response_chunk_file = open(response_chunk_name, 'wb')
# response_chunk_file.write(s.content)
# response_chunk_file.close()

b'"350","7020 Ramona Blvd ,  Jacksonville,  FL, 32205","Match","Exact","7020 RAMONA BLVD, JACKSONVILLE, FL, 32205","-81.76221334839232,30.313754951505928","83989119","L","12","031","012000","1000"\n"230","10063 Baymeadows Road ,  Jacksonville,  FL, 32256","No_Match"\n"351","8964 Old Plank Road ,  Jacksonville,  FL, 32220","Match","Exact","8964 OLD PLANK RD, JACKSONVILLE, FL, 32220","-81.80266744033486,30.330543854852237","83957169","L","12","031","011901","2004"\n"110","700 Seagate Ave,  Neptune Beach, Fl, 32266-3505","Match","Non_Exact","700 SEAGATE AVE, NEPTUNE BEACH, FL, 32266","-81.40182204301527,30.3076187264042","83922963","R","12","031","014001","3003"\n"231","9000 Southside Blvd. Bldg. 900,  Jacksonville,  FL, 32256","Match","Exact","9000 SOUTHSIDE BLVD, JACKSONVILLE, FL, 32256","-81.5518212506077,30.195179698600384","83952442","L","12","031","015922","2008"\n"352","2841 Alaskan Way ,  Jacksonville,  FL, 32226","Match","Exact","2841 ALASKAN WAY, JACKSONVILLE, FL, 32226","-81.57

In [23]:
# raw_file = open(response_chunk_name, 'rb+')
dfCensusResults = pd.read_csv(csv_string_readable, sep=',', header=None)
dfCensusResults.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,350,"7020 Ramona Blvd , Jacksonville, FL, 32205",Match,Exact,"7020 RAMONA BLVD, JACKSONVILLE, FL, 32205","-81.76221334839232,30.313754951505928",83989119.0,L,12.0,31.0,12000.0,1000.0
1,230,"10063 Baymeadows Road , Jacksonville, FL, 32256",No_Match,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,351,"8964 Old Plank Road , Jacksonville, FL, 32220",Match,Exact,"8964 OLD PLANK RD, JACKSONVILLE, FL, 32220","-81.80266744033486,30.330543854852237",83957169.0,L,12.0,31.0,11901.0,2004.0
3,110,"700 Seagate Ave, Neptune Beach, Fl, 32266-3505",Match,Non_Exact,"700 SEAGATE AVE, NEPTUNE BEACH, FL, 32266","-81.40182204301527,30.3076187264042",83922963.0,R,12.0,31.0,14001.0,3003.0
4,231,"9000 Southside Blvd. Bldg. 900, Jacksonville,...",Match,Exact,"9000 SOUTHSIDE BLVD, JACKSONVILLE, FL, 32256","-81.5518212506077,30.195179698600384",83952442.0,L,12.0,31.0,15922.0,2008.0


In [27]:
dfCensusResults.columns = ['id', 'ADDRESS_IN', 'MATCH_INDICATOR', 'MATCH_TYPE', 'ADDRESS_OUT', 'LONG_LAT', 'TIGER_EDGE', 'STREET_SIDE', 'FIPS_STATE', 'FIPS_COUNTY', 'CENSUS_TRACT', 'CENSUS_BLOCK']
dfCensusResults.count()

id                 382
ADDRESS_IN         382
MATCH_INDICATOR    382
MATCH_TYPE         366
ADDRESS_OUT        366
LONG_LAT           366
TIGER_EDGE         366
STREET_SIDE        366
FIPS_STATE         366
FIPS_COUNTY        366
CENSUS_TRACT       366
CENSUS_BLOCK       366
dtype: int64

In [25]:
dfCensusResults.head()

,id,ADDRESS_IN,MATCH_INDICATOR,MATCH_TYPE,ADDRESS_OUT,LONG_LAT,TIGER_EDGE,STREET_SIDE,FIPS_STATE,FIPS_COUNTY,CENSUS_TRACT,CENSUS_BLOCK
0,350,"7020 Ramona Blvd , Jacksonville, FL, 32205",Match,Exact,"7020 RAMONA BLVD, JACKSONVILLE, FL, 32205","-81.76221334839232,30.313754951505928",83989119.0,L,12.0,31.0,12000.0,1000.0
1,230,"10063 Baymeadows Road , Jacksonville, FL, 32256",No_Match,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,351,"8964 Old Plank Road , Jacksonville, FL, 32220",Match,Exact,"8964 OLD PLANK RD, JACKSONVILLE, FL, 32220","-81.80266744033486,30.330543854852237",83957169.0,L,12.0,31.0,11901.0,2004.0
3,110,"700 Seagate Ave, Neptune Beach, Fl, 32266-3505",Match,Non_Exact,"700 SEAGATE AVE, NEPTUNE BEACH, FL, 32266","-81.40182204301527,30.3076187264042",83922963.0,R,12.0,31.0,14001.0,3003.0
4,231,"9000 Southside Blvd. Bldg. 900, Jacksonville,...",Match,Exact,"9000 SOUTHSIDE BLVD, JACKSONVILLE, FL, 32256","-81.5518212506077,30.195179698600384",83952442.0,L,12.0,31.0,15922.0,2008.0


In [28]:
outfile = "/opt/work/wooly-tongue/src/jupyter/uscensus/data/SchoolAddressesWithCensus2.csv"

dfCensusResults.to_csv(outfile, index=False, header=True)